In [1]:
import pandas as pd
import numpy as np
import os, sys
import matplotlib.pyplot as plt
import seaborn as sns
import json
sys.path.append('../')
from models.knobs import rocksdb_knobs_make_dict

In [2]:
PATH_internal = "internal"
PATH_external = "external"
PATH_knobs = "rocksdb_conf"
wk_len = 16

In [3]:
knobs = rocksdb_knobs_make_dict(PATH_knobs)
knobs = pd.DataFrame(data=knobs['data'].astype(np.float32), columns=knobs['columnlabels'])
columns = knobs.columns
knobs.head(4)

,max_background_compactions,max_background_flushes,write_buffer_size,max_write_buffer_number,min_write_buffer_number_to_merge,compaction_pri,compaction_style,level0_file_num_compaction_trigger,level0_slowdown_writes_trigger,level0_stop_writes_trigger,...,open_files,block_size,cache_index_and_filter_blocks,max_bytes_for_level_base,max_bytes_for_level_multiplier,target_file_size_base,target_file_size_multiplier,num_levels,memtable_bloom_size_ratio,compression_ratio
0,6.0,16.0,1304576.0,3.0,2.0,1.0,0.0,3.0,24.0,44.0,...,10000.0,9216.0,0.0,5242880.0,11.0,1800192.0,1.0,8.0,0.10,0.67
1,2.0,14.0,911360.0,6.0,2.0,0.0,0.0,7.0,18.0,55.0,...,100000.0,15360.0,1.0,3145728.0,10.0,1729536.0,1.0,5.0,0.15,0.50
2,8.0,6.0,1474560.0,6.0,1.0,2.0,0.0,5.0,18.0,48.0,...,1000000.0,6144.0,0.0,4194304.0,9.0,1429504.0,1.0,6.0,0.05,0.36
3,12.0,8.0,550912.0,7.0,1.0,3.0,0.0,4.0,18.0,35.0,...,10000.0,3072.0,1.0,2097152.0,11.0,900096.0,1.0,7.0,0.05,0.63


In [4]:
PATH_internal = "internal"
PATH_external = "external"
wk_len = 16
internal_dict = {}

pruned_im = pd.read_csv(os.path.join(PATH_internal, 'internal_ensemble_pruned_tmp.csv'), index_col=0)
for wk in range(wk_len):
    im = pd.read_csv(os.path.join(PATH_internal, f'internal_results_{wk}.csv'), index_col=0)
    internal_dict[wk] = im[pruned_im.columns]
#     break
internal_dict[0].head(4)

,rocksdb.block.cache.miss COUNT,rocksdb.block.cache.hit COUNT,rocksdb.block.cache.add COUNT,rocksdb.block.cache.index.miss COUNT,rocksdb.block.cache.index.hit COUNT,rocksdb.block.cache.index.add COUNT,rocksdb.block.cache.index.bytes.insert COUNT,rocksdb.block.cache.data.miss COUNT,rocksdb.block.cache.data.hit COUNT,rocksdb.block.cache.data.add COUNT,...,rocksdb.bytes.decompressed_P99,rocksdb.bytes.decompressed_P100,rocksdb.bytes.decompressed_COUNT,rocksdb.bytes.decompressed_SUM,rocksdb.db.flush.micros_P50,rocksdb.db.flush.micros_P95,rocksdb.db.flush.micros_P99,rocksdb.db.flush.micros_P100,rocksdb.db.flush.micros_COUNT,rocksdb.db.flush.micros_SUM
index,,,,,,,,,,,,,,,,,,,,,
/home/jieun/data_generation_rocksdb/rocksdb_conf/config1.cnf,1296845,1172926,1213729,0,0,0,0,1296845,1172926,1213729,...,8383.000000,8383.0,1297251,10856979511,21652.173913,30615.000000,30615.0,30615.0,44,1015663
/home/jieun/data_generation_rocksdb/rocksdb_conf/config2.cnf,1304820,4192080,1256074,1582,2701880,1582,5078096,1303238,1490200,1254492,...,14659.000000,14659.0,1304820,19074632333,18135.593220,21979.661017,26561.0,26561.0,63,1074316
/home/jieun/data_generation_rocksdb/rocksdb_conf/config3.cnf,1407632,2195141,1315590,0,0,0,0,1407632,2195141,1315590,...,6578.256132,12483.0,1407897,8847109704,29660.714286,45820.000000,45820.0,45820.0,78,2692570
/home/jieun/data_generation_rocksdb/rocksdb_conf/config4.cnf,1497592,7007880,1252738,974,4003817,974,6993856,1496618,3004063,1251764,...,4385.648850,9451.0,1497592,4718848784,24646.616541,32387.969925,34773.0,34773.0,208,5221234


In [5]:
external_dict = {}
for wk in range(wk_len):
    ex = pd.read_csv(os.path.join(PATH_external, f'external_results_{wk}.csv'), index_col=0)
    external_dict[wk] = ex
    break
external_dict[0].head(4)

,TIME,RATE,WAF,SA
index,,,,
/home/jieun/data_generation_rocksdb/rocksdb_conf/config1.cnf,21.8,4.77,8.7,72.17
/home/jieun/data_generation_rocksdb/rocksdb_conf/config2.cnf,25.9,4.02,8.9,55.83
/home/jieun/data_generation_rocksdb/rocksdb_conf/config3.cnf,52.0,2.00,7.5,34.21
/home/jieun/data_generation_rocksdb/rocksdb_conf/config4.cnf,53.6,1.94,10.5,56.94


In [7]:
from scipy.spatial import distance
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [8]:
def euclidean_distance(a, b):
    res = a - b
    res = res ** 2
    res = np.sqrt(res)
#     return res
    return np.average(res)

In [9]:
scaler = MinMaxScaler().fit(pd.concat(internal_dict))

In [10]:
def get_euclidean_distance(wk_num, size):
    wk = []
    for im_d in internal_dict:
        wk.append(scaler.transform(internal_dict[im_d].iloc[:size, :]))
        
    big = 100
    for i in range(16):
        ed = euclidean_distance(wk[wk_num], wk[i])
        if ed<big and wk_num != i: 
            big=ed
            idx = i
        print(f'{i}th \t{ed}')
    print('best similar workload is ',idx, 'th')

In [11]:
get_euclidean_distance(0, 10)

0th 	0.0
1th 	0.030659465307545538
2th 	0.050035021917459156
3th 	0.09884381319190573
4th 	0.07391478760045503
5th 	0.06644499819289489
6th 	0.08369731489318774
7th 	0.13896057435916373
8th 	0.1284858187950077
9th 	0.10322441229696738
10th 	0.11053186049275815
11th 	0.1676038736314744
12th 	0.1572493438035624
13th 	0.12153435183681227
14th 	0.1278307425219381
15th 	0.18817931853335332
best similar workload is  1 th


In [12]:
get_euclidean_distance(0, 50)

0th 	0.0
1th 	0.03126186853763571
2th 	0.05087441456563779
3th 	0.09899243067320787
4th 	0.07249969551429418
5th 	0.06867185075731101
6th 	0.08608121292118724
7th 	0.14023160996117714
8th 	0.1274756617769466
9th 	0.10384685387014124
10th 	0.11223786954657834
11th 	0.16735151686120894
12th 	0.15449689571301392
13th 	0.12253383098870237
14th 	0.12906369854695454
15th 	0.188041795543214
best similar workload is  1 th


In [13]:
get_euclidean_distance(0, 100)

0th 	0.0
1th 	0.03172075401121331
2th 	0.05180735933117723
3th 	0.09946185684599448
4th 	0.0752382728785048
5th 	0.07050504699554071
6th 	0.0866993523679743
7th 	0.1403915456350811
8th 	0.12975258963517614
9th 	0.10593308637902243
10th 	0.11412389928379277
11th 	0.16842517191551273
12th 	0.16029563371485425
13th 	0.12734910294202997
14th 	0.13097017630491287
15th 	0.18930803433182114
best similar workload is  1 th


In [14]:
get_euclidean_distance(0, 1000)

0th 	0.0
1th 	0.03154428606289283
2th 	0.051781566936908996
3th 	0.09992325639520226
4th 	0.07475300703775677
5th 	0.07024546931020303
6th 	0.08677452841603132
7th 	0.14093863430320086
8th 	0.12994746684099498
9th 	0.10584297449687327
10th 	0.1140831032542511
11th 	0.16940406246150502
12th 	0.15891027042974773
13th 	0.1269718152653781
14th 	0.13137011194389622
15th 	0.19075929537725314
best similar workload is  1 th
